In [ ]:
# Libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import time
import random
import sklearn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
from torch import nn
from torch import optim
from tqdm import tqdm
import torch.nn.functional as F

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Remove printing error
pd.options.mode.chained_assignment = None

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from sklearn import metrics
from sklearn.metrics import roc_auc_score

from functions_folder.nn_MIMICDataset import MIMICDataset

import optuna
from optuna.trial import TrialState
import torch.optim as optim

In [3]:
# Set device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

# Set the random seeds for deterministic results.
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

GPU is available


In [5]:
# Data
mimic_data = pd.read_csv('mimic_data.csv')


# Split into 'preprocessing dataset' and a 'train/vaid/test dataset'
stays = mimic_data['stay_id'].unique()
import random
random.Random(0).shuffle(stays)
model_data2 = mimic_data.set_index("stay_id").loc[stays].reset_index()
n = round(0.5 * len(stays))
preprocessing_stays = stays[:n]
hold_out_stays = stays[n:]
preprocessing_data = model_data2[model_data2['stay_id'].isin(preprocessing_stays)]
hold_out_data = model_data2[model_data2['stay_id'].isin(hold_out_stays)]
preprocessing_data.drop(columns=['iv_treatment_length'], inplace=True)
preprocessing_data

,stay_id,date,iv_flag,first_po_flag,po_flag,blood pressure diastolic0,blood pressure diastolic1,blood pressure diastolic2,blood pressure diastolic3,blood pressure diastolic4,...,temperature14_current_stay_difference,temperature15_current_stay_difference,temperature16_current_stay_difference,temperature17_current_stay_difference,temperature18_current_stay_difference,temperature19_current_stay_difference,temperature20_current_stay_difference,temperature21_current_stay_difference,temperature_mean_current_stay_difference,temperature_std_current_stay_difference
0,39409066.0,2137-05-22,1.0,1.0,0,0.575521,0.519798,0.272727,0.700000,0.0,...,0.6,0.710059,0.600000,0.350276,-1.000000,-1.000000,0.467482,0.4,0.365151,0.075809
1,39409066.0,2137-05-23,1.0,1.0,0,0.448568,0.454810,0.454545,0.411765,0.0,...,0.4,0.410488,0.600000,0.551909,0.500693,0.439452,0.565014,0.4,0.482215,0.321606
2,30322661.0,2169-10-09,1.0,NaN,0,0.432292,0.446094,0.909091,0.304348,0.0,...,0.4,0.396435,0.600000,0.437195,0.526688,0.561250,0.666566,0.4,0.432688,0.363971
3,30322661.0,2169-10-10,1.0,1.0,0,0.361979,0.375081,0.545455,0.304348,0.0,...,0.4,0.396435,0.600000,0.437195,0.526688,0.561250,0.666566,0.4,0.432688,0.363971
4,30322661.0,2169-10-11,1.0,1.0,0,0.397135,0.378309,0.181818,0.500000,0.0,...,1.0,0.388391,0.642857,0.437195,0.526688,0.561250,0.358257,1.0,0.432688,0.664467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14515,37575956.0,2153-02-15,1.0,1.0,0,0.479167,0.536475,0.090909,0.875000,0.0,...,0.6,0.568117,0.600000,0.479335,-1.000000,-1.000000,0.454375,0.4,0.295362,0.681000
14516,37575956.0,2153-02-16,1.0,1.0,0,0.522135,0.525500,0.272727,1.000000,0.0,...,0.6,0.568117,0.600000,0.479335,-1.000000,-1.000000,0.454375,0.4,0.295362,0.681000
14517,37540542.0,2161-05-10,1.0,NaN,0,0.253906,0.237573,0.090909,-1.000000,0.0,...,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000
14518,37540542.0,2161-05-11,1.0,1.0,0,0.397135,0.370562,0.363636,0.304348,0.0,...,0.6,0.562664,0.528571,-1.000000,-1.000000,-1.000000,-1.000000,0.4,0.234579,0.425554


In [6]:
all_shap_importance = pd.read_csv('all_shap_importance.csv')

shap_list = all_shap_importance[all_shap_importance['feature_importance_vals'] >= 0.5].col_name.unique().tolist()
shap_list = ['stay_id', 'date', 'iv_flag', 'first_po_flag', 'po_flag'] + shap_list
preprocessing_data = preprocessing_data[shap_list]
preprocessing_data

,stay_id,date,iv_flag,first_po_flag,po_flag,blood pressure systolic17_current_stay,heart rate17,respiratory rate11_current_stay,blood pressure mean11_current_stay,o2 saturation pulseoxymetry3_current_stay,...,blood pressure systolic12_current_stay,o2 saturation pulseoxymetry1_current_stay_difference,gcs - verbal response14_current_stay,temperature7_current_stay,temperature6_current_stay_difference,respiratory rate3,heart rate21_current_stay,o2 saturation pulseoxymetry7_difference,gcs - verbal response2_difference,heart rate8
0,39409066.0,2137-05-22,1.0,1.0,0,0.966557,-1.000000,0.761556,0.762332,1.000000,...,0.222222,0.542862,0.0,0.000000e+00,0.4,0.633333,0.000000,0.491376,-1.0,0.384615
1,39409066.0,2137-05-23,1.0,1.0,0,0.431357,0.343572,0.126674,0.539524,0.573477,...,0.333333,0.599999,0.0,1.083738e-28,0.4,0.745098,0.111111,0.279022,-1.0,0.615385
2,30322661.0,2169-10-09,1.0,NaN,0,0.414066,0.230392,0.095854,0.400227,0.583127,...,0.111111,0.599999,0.0,9.099612e-29,0.4,0.527778,0.148148,0.364522,-1.0,0.076923
3,30322661.0,2169-10-10,1.0,1.0,0,0.327966,0.221895,0.046076,0.280141,0.460829,...,0.333333,0.599999,0.0,0.000000e+00,0.4,0.275362,0.259259,0.364522,-1.0,0.153846
4,30322661.0,2169-10-11,1.0,1.0,0,0.248910,0.521848,0.032439,0.198931,0.430108,...,0.333333,0.599999,0.0,1.622159e-01,0.4,0.452381,0.592593,0.571676,-1.0,0.230769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14515,37575956.0,2153-02-15,1.0,1.0,0,0.988171,-1.000000,0.953186,0.569251,-1.000000,...,0.000000,0.142857,-1.0,-1.000000e+00,0.4,0.791667,0.000000,0.439876,0.0,0.230769
14516,37575956.0,2153-02-16,1.0,1.0,0,0.626321,-1.000000,0.166640,0.800980,0.774194,...,0.111111,0.142857,-1.0,-1.000000e+00,0.4,0.904762,0.111111,0.439876,0.0,0.538462
14517,37540542.0,2161-05-10,1.0,NaN,0,0.968004,-1.000000,0.761556,0.718315,-1.000000,...,0.111111,-1.000000,-1.0,-1.000000e+00,-1.0,-1.000000,0.000000,-1.000000,-1.0,-1.000000
14518,37540542.0,2161-05-11,1.0,1.0,0,0.606376,0.521273,0.387155,0.412453,0.564516,...,0.333333,0.085715,0.0,2.895331e-29,-1.0,0.263889,0.259259,-1.000000,-1.0,0.307692


In [7]:
individual = '110111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [8]:
stays = preprocessing_data['stay_id'].unique()
import random
random.Random(5).shuffle(stays) # Note changed random split to 5 so split between po_flag is similar for val and test sets 
X_data = preprocessing_data.drop(columns=['stay_id', 'po_flag', 'date', 'iv_flag', 'first_po_flag'])
# Filter for features in this individual
X_data = X_data.loc[:, [True if individual[i] == '1' else False for i in range(len(individual))]]
model_data = pd.concat([preprocessing_data[['stay_id', 'po_flag']], X_data], axis=1)
model_data2 = model_data.set_index("stay_id").loc[stays].reset_index()
n = round(0.7 * len(stays))
n2 = round(0.85 * len(stays))
train_stays = stays[:n]
validation_stays = stays[n:n2]
test_stays = stays[n2:]
train_data = model_data2[model_data2['stay_id'].isin(train_stays)]
valid_data = model_data2[model_data2['stay_id'].isin(validation_stays)]
test_data = model_data2[model_data2['stay_id'].isin(test_stays)]

In [9]:
preprocessing_data.po_flag.value_counts(normalize=True)
hold_out_data.po_flag.value_counts(normalize=True)
len(train_data)
train_data.po_flag.value_counts(normalize=True)
len(valid_data)
valid_data.po_flag.value_counts(normalize=True)
len(test_data)
test_data.po_flag.value_counts(normalize=True)

0    0.836019
1    0.163981
Name: po_flag, dtype: float64

0    0.83458
1    0.16542
Name: po_flag, dtype: float64

10207

0    0.840404
1    0.159596
Name: po_flag, dtype: float64

2127

0    0.827927
1    0.172073
Name: po_flag, dtype: float64

2186

0    0.823422
1    0.176578
Name: po_flag, dtype: float64

In [10]:
# Oversampling train set
from typing import Counter
import imblearn
from imblearn.over_sampling import SMOTE
# Split X y
train_data_X = train_data.drop(columns=['stay_id', 'po_flag'])
train_data_y = train_data['po_flag']
Counter(train_data_y)
oversample = SMOTE()
train_data_X, train_data_y = oversample.fit_resample(train_data_X, train_data_y)
Counter(train_data_y)
train_data_y = pd.DataFrame(train_data_y, columns=['po_flag'])
train_data_y['stay_id'] = 'x'
train_data = pd.concat([train_data_y, train_data_X], axis=1)
train_data = train_data.sample(frac=1, random_state=0).reset_index(drop=True)

Counter({0: 8578, 1: 1629})

Counter({0: 8578, 1: 8578})

Optimize

In [22]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    in_features = 5
    layers.append(nn.BatchNorm1d(in_features))
    p = trial.suggest_categorical("dropout", [0, 0.1, 0.2, 0.3, 0.4, 0.5])

    for i in range(n_layers):
        out_features = trial.suggest_categorical("n_units_l{}".format(i), [2, 4, 8, 16, 32, 64, 128, 256, 512])
        #layers.append(nn.BatchNorm1d(in_features))
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        #p = trial.suggest_float("dropout_l{}".format(i), 0, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, 1))

    return nn.Sequential(*layers)

In [23]:
def objective(trial):

    # Generate the model.
    model = define_model(trial).to(device)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam"])
    #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    lr = trial.suggest_categorical("lr", [0.001, 0.0001])
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    batch_size = trial.suggest_categorical("batch_size", [256, 512])

    # Get the dataset.
    train_dataset = MIMICDataset(train_data)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, collate_fn=train_dataset.collate_fn_padd)

    valid_dataset = MIMICDataset(valid_data)
    valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=batch_size, collate_fn=valid_dataset.collate_fn_padd)

    #test_dataset = MIMICDataset(test_data)
    #test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, collate_fn=test_dataset.collate_fn_padd)

    # Define loss
    criterion = nn.BCEWithLogitsLoss()

    best_valid_auroc = 0

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        epoch_loss = 0
        batch_prediction_list = []
        batch_label_list = []
        for batch_idx, (labels, features) in enumerate(train_dataloader):
            features = features.float()
            labels = labels.float()
            features = features.to(device=device)
            labels = labels.to(device=device)
            optimizer.zero_grad()
            features2 = torch.permute(features, (1, 0))
            labels2 = torch.permute(labels, (1, 0))
            output = model(features2)
            loss = criterion(output, labels2)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            sig = torch.nn.Sigmoid()
            output = sig(output)      
            np_predictions = output.cpu().detach().numpy()
            np_labels = labels2.cpu().detach().numpy()
            np_predictions = np_predictions.squeeze()
            np_labels = np_labels.squeeze()
            np_predictions = np_predictions.flatten()
            np_labels = np_labels.flatten()
            for x in np_predictions:
                batch_prediction_list.append(x)
            for x in np_labels:
                batch_label_list.append(x)
        final_predictions = np.array(batch_prediction_list)
        final_labels = np.array(batch_label_list)
        train_auroc = roc_auc_score(final_labels, final_predictions.round())
        train_loss = epoch_loss / len(train_dataloader)
        print('epoch:', epoch)
        print('train_auroc:', train_auroc)
        print('train_loss:', train_loss)

        # Validation of the model.
        model.eval()
        epoch_loss = 0
        batch_prediction_list = []
        batch_label_list = []
        with torch.no_grad():
            for batch_idx, (labels, features) in enumerate(valid_dataloader):
                features = features.float()
                labels = labels.float()
                features = features.to(device=device)
                labels = labels.to(device=device)
                features2 = torch.permute(features, (1, 0))
                labels2 = torch.permute(labels, (1, 0))
                output = model(features2)
                loss = criterion(output, labels2)
                epoch_loss += loss.item()
                sig = torch.nn.Sigmoid()
                output = sig(output)  
                np_predictions = output.cpu().detach().numpy()
                np_labels = labels2.cpu().detach().numpy()
                np_predictions = np_predictions.squeeze()
                np_labels = np_labels.squeeze()
                np_predictions = np_predictions.flatten()
                np_labels = np_labels.flatten()
                # Create list
                for x in np_predictions:
                    batch_prediction_list.append(x)
                for x in np_labels:
                    batch_label_list.append(x)
        final_predictions = np.array(batch_prediction_list)
        final_labels = np.array(batch_label_list)
        valid_auroc = roc_auc_score(final_labels, final_predictions.round())
        valid_loss = epoch_loss / len(train_dataloader)
        print('valid_auroc:', valid_auroc)
        print('valid_loss:', valid_loss)
        #print(type(valid_auroc))

        trial.report(valid_auroc, epoch)

        if valid_auroc > best_valid_auroc:
            best_valid_auroc = valid_auroc

        if valid_auroc == 0.5:
            raise optuna.exceptions.TrialPruned()

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_valid_auroc

In [24]:
EPOCHS = 10
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics:")
print("Number of finished trials:", len(study.trials))
print("Number of pruned trials:", len(pruned_trials))
print("Number of complete trials:", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("Value:", trial.value)

print("Params:")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-10-13 17:16:53,792] A new study created in memory with name: no-name-7f18290c-15fb-47f6-985e-86d45073ffc9


epoch: 0
train_auroc: 0.6608766612263931
train_loss: 0.6504264842061436
valid_auroc: 0.7537353031530147
valid_loss: 0.07869259750141817
epoch: 1
train_auroc: 0.7372347866635578
train_loss: 0.5928251769612817
valid_auroc: 0.7688952811833814
valid_loss: 0.07068945116856519
epoch: 2
train_auroc: 0.7546048029843786
train_loss: 0.5520569515578887
valid_auroc: 0.776330978114149
valid_loss: 0.06705102455966613
epoch: 3
train_auroc: 0.7571695033807414
train_loss: 0.5267663764602998
valid_auroc: 0.7850792675547611
valid_loss: 0.06414275835542117
epoch: 4
train_auroc: 0.7656796456050361
train_loss: 0.5092424149898922
valid_auroc: 0.7827380121205351
valid_loss: 0.06265716955942266
epoch: 5
train_auroc: 0.7678946141291676
train_loss: 0.5002565883538302
valid_auroc: 0.7838737304623862
valid_loss: 0.06188349644927418
epoch: 6
train_auroc: 0.7688272324551177
train_loss: 0.4916898132685353
valid_auroc: 0.7814789473194254
valid_loss: 0.06110561463762732
epoch: 7
train_auroc: 0.7717416647237119
train_lo

[I 2022-10-13 17:19:37,902] Trial 0 finished with value: 0.7877253981996071 and parameters: {'n_layers': 2, 'dropout': 0.3, 'n_units_l0': 256, 'n_units_l1': 16, 'optimizer': 'Adam', 'lr': 0.0001, 'batch_size': 256}. Best is trial 0 with value: 0.7877253981996071.


valid_auroc: 0.7877253981996071
valid_loss: 0.05979804212556166
epoch: 0
train_auroc: 0.6289927722079738
train_loss: 0.6494898357812096
valid_auroc: 0.7636728386442129
valid_loss: 0.07067236435763977
epoch: 1
train_auroc: 0.7427139193285147
train_loss: 0.5331066614564728
valid_auroc: 0.7797381331397026
valid_loss: 0.06031613183372161
epoch: 2
train_auroc: 0.7663791093494986
train_loss: 0.493653845261125
valid_auroc: 0.7797753698066485
valid_loss: 0.05933475406730876
epoch: 3
train_auroc: 0.772149685241315
train_loss: 0.4823207999853527
valid_auroc: 0.783466454417665
valid_loss: 0.059114072690991795
epoch: 4
train_auroc: 0.7738400559570996
train_loss: 0.4781784976229948
valid_auroc: 0.7843717708827883
valid_loss: 0.05909575697253732
epoch: 5
train_auroc: 0.7772790860340406
train_loss: 0.4700102043502471
valid_auroc: 0.7838574394205976
valid_loss: 0.05926600975148818
epoch: 6
train_auroc: 0.780018652366519
train_loss: 0.47285372720045205
valid_auroc: 0.7866967352752255
valid_loss: 0.0594

[I 2022-10-13 17:22:29,052] Trial 1 finished with value: 0.7893219202949144 and parameters: {'n_layers': 3, 'dropout': 0.2, 'n_units_l0': 16, 'n_units_l1': 8, 'n_units_l2': 64, 'optimizer': 'Adam', 'lr': 0.001, 'batch_size': 256}. Best is trial 1 with value: 0.7893219202949144.


...

valid_auroc: 0.7391664572104152
valid_loss: 0.08918156693963443
epoch: 0
train_auroc: 0.570354394963861
train_loss: 0.6883497597540126


[I 2022-10-13 18:20:16,987] Trial 48 pruned. 


valid_auroc: 0.6831764738738236
valid_loss: 0.09274618415271535
epoch: 0
train_auroc: 0.7726159944042901
train_loss: 0.4749483864973573
valid_auroc: 0.7900875992589903
valid_loss: 0.05647086352109909
epoch: 1
train_auroc: 0.7884705059454419
train_loss: 0.44897402866798286
valid_auroc: 0.7935273363681217
valid_loss: 0.05801105060998131
epoch: 2
train_auroc: 0.789869433434367
train_loss: 0.4441017920480055
valid_auroc: 0.7997365505813575
valid_loss: 0.05893013231894549
epoch: 3
train_auroc: 0.7912683609232922
train_loss: 0.44086693599820137
valid_auroc: 0.8013167816348759
valid_loss: 0.05772308304029353
epoch: 4
train_auroc: 0.7902191653065982
train_loss: 0.4392750307041056
valid_auroc: 0.8025758464359856
valid_loss: 0.05766112532685785
epoch: 5
train_auroc: 0.7913849382140359
train_loss: 0.4384562776369207
valid_auroc: 0.8011026707999367
valid_loss: 0.05805331161793541
epoch: 6
train_auroc: 0.791326649568664
train_loss: 0.43649466686389027
valid_auroc: 0.8009956153824671
valid_loss: 0.0

[I 2022-10-13 18:22:57,651] Trial 49 finished with value: 0.802929594771972 and parameters: {'n_layers': 2, 'dropout': 0.2, 'n_units_l0': 512, 'n_units_l1': 512, 'optimizer': 'Adam', 'lr': 0.001, 'batch_size': 256}. Best is trial 24 with value: 0.8033368708166932.


valid_auroc: 0.7985635955725603
valid_loss: 0.05997050027636921
Study statistics:
Number of finished trials: 50
Number of pruned trials: 30
Number of complete trials: 20
Best trial:
Value: 0.8033368708166932
Params:
    n_layers: 2
    dropout: 0.4
    n_units_l0: 512
    n_units_l1: 16
    optimizer: Adam
    lr: 0.001
    batch_size: 256
